## Develop a factory manager (agent) for the SCM world

This tutorial describes how to develop an agent for the SCM world, test it, and submit it to the ANAC 2019 SCM league.

The first step is to install negmas

```bash
pip install negmas
```
Once you have this library installed, you can start developing your factory manager:

In [1]:
from negmas.apps.scml import FactoryManager

class MyFactoryManager(FactoryManager):
    """My factory manager"""

Try to make an object of your newly created factory manager. We will fail, do not panic:

In [2]:
try:
    incorrect = FactoryManager()
except TypeError as e:
    print(e)

Can't instantiate abstract class FactoryManager with abstract methods on_negotiation_request


You are told that you cannot instantiate your newly created class as did not implement the abstract (required) method ```on_negotiation_request```. That is actually the ONLY abstract method you need to implement.

In [3]:
class MyFactoryManager(FactoryManager):
    """My factory manager"""
    
    def on_negotiation_request(self, cfp: "CFP", partner: str):
        return None

As the [documentation](http://www.yasserm.com/negmas/negmas.apps.scml.html?highlight=on_negotiation_request#negmas.apps.scml.SCMLAgent.on_negotiation_request) states, this function is called whenever your factory manager receives a request from another agent to negotiate. You can either return `None` if you do not want to accept this negotiation or create  a `Negotiator` that represents your agent in it.


Your do-nothing agent is almost ready. Let's try it:

In [4]:
from negmas.apps.scml import anac2019_world

world = anac2019_world(competitors=(MyFactoryManager,), n_steps=5)

You have now a world with the settings for ANAC 2019 competition. Let's run it

In [5]:
world.run()

What just happened?

In [6]:
world.stats

defaultdict(list,
            {'n_cfps_on_board_before': [0, 20, 13, 3, 0],
             'n_registered_negotiations_before': [0, 0, 0, 0, 0],
             'n_contracts_executed': [0, 0, 0, 0, 0],
             'n_contracts_cancelled': [0, 40, 0, 0, 0],
             'n_breaches': [0, 0, 0, 5, 5],
             'breach_level': [-1, -1, -1, 1.0, 1.0],
             'n_contracts_signed': [0, 10, 0, 0, 0],
             'n_contracts_concluded': [50, 0, 0, 0, 0],
             'n_negotiations': [50, 0, 0, 0, 0],
             'n_registered_negotiations_after': [0, 0, 0, 0, 0],
             'activity_level': [0, 0, 0, 0, 0],
             'n_cfps': [25, 21, 0, 0, 0],
             'n_cfps_on_board_after': [20, 13, 3, 0, 0],
             '_balance_bank': [0.0, 0.0, 0.0, 0.0, 0.0],
             '_balance_society': [0.0, 0.0, 0.0, 5.0, 10.0],
             '_balance_insurance': [0.0, 0.0, 0.0, 0.0, 0.0],
             '_storage_insurance': [0, 0, 0, 0, 0],
             'balance_m_0': [0.0, 0.0, 0.0, 0.0, 

The property `stats` in `World` gives you several statistics about the world for every time-step of the simulation.

Let's try to run a tournament with this do-nothing agent against the built-in greedy agent:

In [7]:
from negmas.apps.scml import anac2019_tournament

results = anac2019_tournament(competitors=(MyFactoryManager,)
                              , n_runs=4, n_steps=5, parallelism='serial')

You can see the scores that each individual factory manager got:

In [8]:
results.scores

,log_file,name,score,type,world
0,None,greedy_1_0,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
1,None,greedy_1_1,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
2,None,greedy_2_0,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
3,None,greedy_2_1,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
4,None,greedy_2_2,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
5,None,greedy_2_3,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
6,None,greedy_3_0,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
7,None,greedy_3_1,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
8,None,greedy_3_2,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2
9,None,greedy_3_3,0.0,greedy,SCMLWorld_95f7d541-bbab-46df-b929-b4796878bdc2


You can also check the total scores for each factory manager type:

In [9]:
results.total_scores

type
greedy    2878.0
my         367.0
Name: score, dtype: float64

If you want, you can check if these differences are statistically significant using a t-test:

In [10]:
results.ttest

,a,b,p,t
0,greedy,my,0.041044,2.068314


So the greedy factory manager is better than the do-nothing factory manager getting a total score of 2878 compared with only 367 for the do-nothing factory manager. Moreover, this difference is statistically significant as the p-value is 0.041 < 0.05.

Or, let's just find out who was the winner:

In [11]:
results.winner

'greedy'

and what was its to score:

In [12]:
results.winner_score

2878.0

Now, you completed the development of your factory manager, tested it by running it in worlds and tournaments, what exactly should you do to participate in the SCM league @ ANAC 2019:

You need to submit the following items:

- Names of all members of the team with their affiliations and email addresses
- Either a single python file with the whole implementation of your agent with any supporting code or a zip file with a single folder containing your code. In the later case, you will need to indicate the class name of your factory manager. Any factory manager names are accepted except (Insurance, Bank, MFactoryManager, CFactoryManager).
- A 2-pages academic report about your factory manager. Please check the submission website for details about this report.

That is it folks!
You can now start developing your own factory manager. Have fun.